In [1]:
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from PIL import Image
import numpy as np
import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.backend as K
from tensorflow.keras import Model
import random
from tensorflow.keras.utils import to_categorical

config = tf.ConfigProto()  
config.gpu_options.allow_growth=True
session = tf.Session(config=config)
K.set_session(session)

In [2]:
def save_model_to_serving(inputs, outputs, export_version, export_path):

    inputs = tf.saved_model.utils.build_tensor_info(inputs)

    outputs = tf.saved_model.utils.build_tensor_info(outputs)
    
    signature = tf.saved_model.signature_def_utils.build_signature_def(                                                                        
        inputs={'x': inputs}, 
        outputs={'scores':outputs},
        method_name=tf.saved_model.signature_constants.PREDICT_METHOD_NAME)
    
    export_path = export_path+'/'+export_version
    builder = tf.saved_model.builder.SavedModelBuilder(export_path)
    main_op = tf.group(tf.tables_initializer(), name='main_op')
    builder.add_meta_graph_and_variables(
        sess=K.get_session(),                                                                                                                    
        tags=[tf.saved_model.tag_constants.SERVING],                                                                                             
        signature_def_map={                                                                                                                      
            'classify_image': signature,                                                                                                                     
        },
        main_op=main_op)
    builder.save()

In [3]:
def get_inputs(param):
    my_input = tf.map_fn(elems=param, fn=tf.image.decode_jpeg, dtype=tf.uint8)
    my_input = K.reshape(my_input,[-1, 224,224,3])
    my_input = K.cast(my_input, dtype=tf.float32)
    my_input = tf.divide(my_input, tf.constant(127.5))
    my_input = tf.subtract(my_input, tf.constant(1.0))
    return my_input


In [4]:
K.set_learning_phase(0)
my_input = keras.layers.Input(shape=[],dtype=tf.string)
my_input = keras.layers.Lambda(get_inputs, output_shape=(224,224,3))(my_input)

base_model = MobileNetV2(alpha=1.4, include_top=False, input_tensor=my_input, weights=None)


inputs = base_model.input
x = base_model.output
x = keras.layers.GlobalAveragePooling2D()(x)
y = keras.layers.Dense(2, activation='softmax')(x)

model = Model(inputs = inputs, outputs = y)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.load_weights('mobilenet_best2.h5')

save_model_to_serving(model.input, model.output, '1111', './mnet')

INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: ./mobilenet/1111/saved_model.pb
